This tutorial can be found and ran in the GITHUB libray corrAI: https://github.com/BuildingEnergySimulationTools/corrai

---
# Characterisation method tutorial

The aim of this tutorial is to provide tools for sensitivity analyses of models using **CorrAI** modules. It contains several parts: 

1. **Model definition**: presentation of the test case used here, and definition of a mathematical model describing it.
  
2. **Measurement import**: Loading of measurements performed on a test benchto serve as reference data for comparison with the model.

3. **Performing sensitivity analysis**: Sensitivity analysis conducted to identify materials properties which have an influence on the discrepancy between model outputs and measured phenomenon
---


# 1. Model definition

## Use case presentation

A "real-scale" test bench is used. The **O3BET** (or in this example BEF test bench in Anglet, France) offers experimental conditions to evaluate building façade solutions. Heat exchanges in a cell are restricted on five of its faces, while the sixth face is dedicated to the tested solution. Internal temperature and humidity conditions can be controlled or monitored. External conditions, including temperatures and solar radiation, are measured.

The tested technology here is a green façade, coupled wiht insulated panels. The experimental setup is presented in the following picture: one cell is equiped with the technology (right one) and another serves as a reference (insulation only).

<div style="text-align: center;">
  <img src="images/cladding.png" style="height:200px; margin-right:10px;">
  <img src="images/BEF_facades.jpeg" style="height:200px;"><br>
 <em>Figure : Pictures of reference wall and vegatal wall installation </em>
</div>

Sensors (heatflux density meters, thermocouples, RTD) are positioned in several parts: in the middle of insulation panels, between insulation and concrete layers, between leaves, in substrates, indoor. Climatic conditions (external temperature, incident solar radiation) are also monitored.


<div style="text-align: center;">
  <img src="images/sensors.png" style="height:300px;"><br>
  <em>Figure : Sensors installation scheme</em>
</div>

- Measure campaign spans from  april 2024 to october 2024
- Acquisition timestep is 60 secondes minimum.

## Description of the proposed model 

The following framework is proposed to identify the **REFERENCE** wall thermal conductivity.

For this example we propose a resistance/capacity approach. Based on electrical circuit analogy, each layer of the wall is modeled by two resistances and a capacity:


| Figure : RC model|
| :---: |
| <img src="images/RC_model.png"  style="height:400px;">   | 


Note that it is recommended to start with a **very simple** model (e.g., one resistance, one capacity with only Text and Tint as boundary conditions) and gradually make it more complex as you identify parameters. For the sake of this tutorial, the proposed model is already a bit detailed.


The following is a brief description of the thermal model:

- Each wall layer is modeled by 2 thermal resistances and a capacity.
    - Resistances to create a gradiant and better resolution of distribution of heat flow : $ R_1 = R_2 = \frac{ep_{layer}}{lambda_{layer} \times 2} $ 
    - Capacity in the middle of both our layers, representing its thermal mass and ability to store heat. : $ C = ep_{layer} \times rho_{layer} \times cap_{layer} $
 
- Inside and outside convection/conduction transfers are model as a constant value thermal resistance.

- Infrared transfers are considered :
    - With the sky, with $ T_{sky} = 0.0552T_{ext}^{1.5} $ as the sky is a significant source of infrared radiation, especially at night. This radiation can have a considerable impact on the thermal behavior of the system, influencing both heating and cooling processes
    - With the surrounding considered to be at $ T_{ext} $ as surroundings or environment also emit infrared radiation

- Short wave solar radiation heat flux is computed $Sw_{gain} = Pyr \times \alpha_{coat} $ with $Pyr$ the measured solar radiation onthe wall (W/m²) and  $\alpha_{coat}$ the coating solar absorbtion coefficient.

- Temperatures $ T_{ext}$ and $T_{int} $ are boundary conditions.  $ T_{int}$ represents the temperature within the controlled environment of the system.

Here are somes theoretical parameters for the model:

In [ ]:
# Surface of the tested wall
S_wall =  7

# Thickness of layer
ep_concrete = 0.200 #m
ep_ins = 0.140 #m

# Conductivity of layer
lambda_concrete = 0.13 # (W/mK)
lambda_ins = 0.031 # W/(mK)

# Density of layer
rho_concrete = 2400 # kg/m3 
rho_ins = 32.5  # kg/m3"


sc_concrete = 880 # J/kg.K
sc_ins = 1000 # J/kg.K

# solar paremetesr
alpha = 0.2 # absorption coefficient
epsilon = 0.8 # emissivity 
fview = 0.5 # view factor of tested wall

## Defining a Simulator in corrAI

The  following `OpaqueWallSimple` class implements a simplified wall heat transfer model, written purely in Python, and compatible with corrAI. It follows the `Model` interface, which guarantees that it can be used in calibration, sensitivity analysis, and optimization workflows and ensures a common interface across different model types: Pure Python models, FMU models (via `ModelicaFmuModel`), Modelica models (via [modelitool](https://github.com/BuildingEnergySimulationTools/modelitool))...

A valid simulator must implement the following concepts:

- **`simulate(property_dict, simulation_options, simulation_kwargs)`**
  Main entry point. Runs the simulation and returns results as a `pandas.DataFrame` indexed by time.

- **`property_dict`**
  Dictionary of model parameters `{property_name: value}`.
  These overwrite default values to perform calibration, parameter sweeps, or optimization.
Here, the wall is described by default physical properties, which can be overridden at runtime :

| Parameter      | Description                                | Unit   |
|----------------|--------------------------------------------|--------|
| **R_ext**      | External thermal resistance                | K/W    |
| **R_int**      | Internal thermal resistance                | K/W    |
| **R_concrete** | Concrete thermal resistance                | K/W    |
| **R_ins**      | Insulation thermal resistance              | K/W    |
| **C_concrete** | Heat capacity of concrete                  | J/K    |
| **C_ins**      | Heat capacity of insulation                | J/K    |
| **alpha**      | Solar absorption coefficient               | -      |
| **S_wall**     | Wall surface area                          | m²     |
| **epsilon**    | Emissivity                                 | -      |
| **fview**      | View factor to the sky/ambient             | -      |


- **`simulation_options`**
  Dictionary of simulation environment settings (time span, timestep, input data).
  Defines how the model will be executed, and what external conditions are applied. Here, the model requires a dataframe containing environmental inputs:

| Input        | Description                          | Unit  |
|:------------:|:------------------------------------:|:-----:|
| **time_sec** | Simulation time                      |   s   |
| **T_ext**    | Outdoor air temperature              |  °C   |
| **T_int**    | Indoor air temperature               |  °C   |
| **Pyr**      | Solar radiation from pyranometer     | W/m²  |



By always structuring models with these inputs and outputs, any simulator (FMU, Modelica, or Python) can be plugged into the corrAI workflow without rewriting downstream code.

The simulation returns a `pandas.DataFrame` with T_concrete, T_interface, T_ins.

In [ ]:
import numpy as np
import pandas as pd
from corrai.base.model import Model

class OpaqueWallSimple(Model):
    def simulate(
        self,
        property_dict: dict,
        simulation_options: dict,
        simulation_kwargs: dict | None = None,
    ) -> pd.DataFrame:

        default_parameters = {
            "R_ext": 0.005,
            "R_int": 0.01,
            "R_concrete": 0.10,
            "R_ins": 0.32,
            "C_concrete": 2.95e6,
            "C_ins": 3.64e4,
            "alpha": 0.2,
            "S_wall": 7,
            "epsilon": 0.4,
            "fview": 0.5,
        }
        parameters = {**default_parameters, **property_dict}

        R_ext       = parameters["R_ext"]
        R_int       = parameters["R_int"]
        R_concrete  = parameters["R_concrete"]
        R_ins       = parameters["R_ins"]
        C_concrete  = parameters["C_concrete"]
        C_ins       = parameters["C_ins"]
        alpha       = parameters["alpha"]
        S_wall      = parameters["S_wall"]
        epsilon     = parameters["epsilon"]
        fview       = parameters["fview"]

        sigma = 5.67e-8  # W/m^2/K^4

        df = simulation_options["dataframe"]
        time  = df["time_sec"].values
        T_ext = df["T_ext"].values
        T_int = df["T_int"].values
        Q_rad = df["Pyr"].values

        startTime = simulation_options.get("startTime", time[0])
        stopTime  = simulation_options.get("stopTime",  time[-1])

        mask  = (time >= startTime) & (time <= stopTime)
        time  = time[mask]
        T_ext = T_ext[mask]
        T_int = T_int[mask]
        Q_rad = Q_rad[mask]

        # init
        T_se        = np.zeros(len(time))
        T_concrete  = np.zeros(len(time))
        T_ins       = np.zeros(len(time))
        T_interface = np.zeros(len(time))
        T_si        = np.zeros(len(time))
        T_sky       = np.zeros(len(time))

        T_se[0]        = T_ext[0]
        T_concrete[0]  = 299 - 273.15
        T_ins[0]       = T_int[0]
        T_interface[0] = (T_ins[0] + T_concrete[0]) / 2
        T_si[0]        = T_int[0]
        T_sky[0]       = T_int[0]

        for t in range(1, len(time)):
            dt = time[t] - time[t - 1]

            T_sky[t] = 0.0552 * (T_ext[t] ** 1.5)

            Q_rad_sky = epsilon * fview * sigma * (T_se[t - 1] ** 4 - T_sky[t] ** 4) * S_wall
            Q_rad_amb = epsilon * fview * sigma * (T_se[t - 1] ** 4 - T_ext[t - 1] ** 4) * S_wall
            Q_rad_dir = Q_rad[t - 1] * alpha * S_wall

            T_se[t] = (
                T_ext[t - 1] / R_ext
                + T_ins[t - 1] / (R_ins / 2)
                + Q_rad_dir - Q_rad_sky - Q_rad_amb
            ) / (1 / R_ext + 1 / (R_ins / 2))

            T_interface[t] = (
                T_ins[t - 1] / (R_ins / 2) + T_concrete[t - 1] / (R_concrete / 2)
            ) / (1 / (R_concrete / 2) + 1 / (R_ins / 2))

            T_si[t] = (
                T_int[t - 1] / R_int + T_concrete[t - 1] / (R_concrete / 2)
            ) / (1 / R_int + 1 / (R_concrete / 2))

            T_ins[t] = T_ins[t - 1] + dt / C_ins * (
                (T_se[t] - T_ins[t - 1]) / (R_ins / 2)
                + (T_interface[t] - T_ins[t - 1]) / (R_ins / 2)
            )

            T_concrete[t] = T_concrete[t - 1] + dt / C_concrete * (
                (T_interface[t] - T_concrete[t - 1]) / (R_concrete / 2)
                + (T_si[t] - T_concrete[t - 1]) / (R_concrete / 2)
            )

        # output
        df_out = pd.DataFrame(
            {
                "T_concrete":  T_concrete,
                "T_interface": T_interface,
                "T_ins":       T_ins,
            },
            index=df.index[mask],
        )
        self.simulation_options = simulation_options
        return df_out

We can try to run. Let's create dummy data:
- T_ext: a daily sinusoidal variation around 20°C (e.g., min 15°C at night, max 30°C during the day).
- T_int: more stable indoor temperature (e.g. around 22°C ±1).
- Pyr: global solar radiation in the form of a bell curve (0 at night, max around midday at ~800 W/m²).

In [ ]:
import numpy as np
import pandas as pd

def create_typical_input_df(day_seconds: int = 86400, step: int = 300) -> pd.DataFrame:
    time = np.arange(0, day_seconds + step, step)
    T_ext = 22.5 + 7.5 * np.sin(2 * np.pi * (time / day_seconds - 0.25))
    T_int = 22 + 0.5 * np.sin(2 * np.pi * time / day_seconds)
    Pyr = 800 * np.maximum(0, np.sin(np.pi * time / day_seconds))

    df = pd.DataFrame(
        {
            "T_ext": T_ext,
            "T_int": T_int,
            "Pyr": Pyr,
            "Pyr": Pyr,
        },
        index=pd.Index(time, name="time_sec")
    )
    df["time_sec"]=df.index
    return df
df_typical = create_typical_input_df()
df_typical.head()

Now, let's define: 
- **simulation options**, with start-time, end-time, and a dataframe for boundary conditions
- **a dictionary**, containing values for our different parameters

In [ ]:
sim_opt ={
    "dataframe":df_typical,
    "startTime": df_typical.index[0],
    "endTime": df_typical.index[-1],  
}

In [ ]:
param_dict = {
    "R_ext": 0.04/S_wall,       
    "R_int": 0.13/S_wall,      
    "R_concrete": 1 / (lambda_concrete / ep_concrete) / 2 / S_wall,   
    "R_ins": 1 / (lambda_ins / ep_ins) / 2 / S_wall, 
    "C_ins": rho_ins*ep_ins*S_wall*sc_ins,  
    "C_concrete": rho_concrete*ep_concrete*S_wall*sc_concrete,       
    "alpha": alpha,       
    "S_wall": S_wall,         
    "epsilon": epsilon,
    'fview': fview
}

In [ ]:
model = OpaqueWallSimple()
model.simulate(
    property_dict = param_dict, 
    simulation_options=sim_opt
)

# 2. Measurement import
Let's now load  the reference cell measurement data on python that will be used as boundary conditions. Note that although the data loaded here should have be cleaned beforhand, it is always important to check it before using it for analyses.

## Loading the dataframe

In [ ]:
import pandas as pd
import numpy as np
import os
import plotly.express as px
from pathlib import Path
pd.options.mode.chained_assignment = None  # default='warn'

TUTORIAL_DIR = Path(os.getcwd()).as_posix()

In [ ]:
reference_df = pd.read_csv(
    Path(TUTORIAL_DIR) / "resources/tuto_data_SA.csv",
    index_col=0,
    sep=";",
    decimal=".",
    parse_dates=True
)

In [ ]:
reference_df.head()

## Check missing or inconsistent data

In [ ]:
reference_df.isna().any().any()

There seems to be no missing data. Let's first plot all data to check if nothing stands out. Although data was cleaned (supposedly, some measurement errors and irregularities might have been missed in the process.

In [ ]:
px.line(reference_df)

Here we can see that one of the temperature sensors installed in the insulation panels stopped measuring on september 7th. Only T_ins_2 will be used as the insulation temperature.

In [ ]:
reference_df.loc[:,"T_ins"] = reference_df['T_ins_2'] # middle of insulation temperature
reference_df.loc[:,"T_int"] =reference_df[['Tint_1', 'Tint_2']].mean(axis=1) # indoor temperature
reference_df.loc[:,"T_interface"] =  reference_df[['T_interface_1', 'T_interface_2']].mean(axis=1) # interface temperature, between insulation and concrete panels

## Run the model with real boundary conditions

Let's rerun the model with real measurements, now. The following simulation period contans a short period with both sunny and cloudy days.

In [ ]:
simulation_df = reference_df.loc["2024-09-04 00:00":"2024-09-09 00:00"]

To create a "time_sec" column used by the model, we can use the following function:

In [ ]:
import datetime as dt

def datetime_to_seconds(index_datetime):
    time_start = dt.datetime(index_datetime[0].year, 1, 1, tzinfo=dt.timezone.utc)
    new_index = index_datetime.to_frame().diff().squeeze()
    new_index.iloc[0] = dt.timedelta(
        seconds=index_datetime[0].timestamp() - time_start.timestamp()
    )
    sec_dt = [elmt.total_seconds() for elmt in new_index]
    return list(pd.Series(sec_dt).cumsum())

In [ ]:
simulation_df["time_sec"] = datetime_to_seconds(simulation_df.index)

sim_opt ={
    "dataframe":simulation_df,
    "startTime":simulation_df["time_sec"].iloc[0],
    "endTime": simulation_df["time_sec"].iloc[-1],  
}

In [ ]:
model = OpaqueWallSimple()
init_res = model.simulate(
    property_dict = param_dict, 
    simulation_options=sim_opt
)

Let's compare results of simulation to measurement:

In [ ]:
init_res.index = init_res.index.tz_localize(None)
init_res = init_res.rename(columns={
    "T_concrete": "T_concrete_PYTHON",
    "T_interface": "T_interface_PYTHON",
    "T_ins": "T_insulation_PYTHON",
})
measure_comp = pd.concat([
    simulation_df[["T_interface", "T_ins"]], 
    init_res[["T_interface_PYTHON", "T_insulation_PYTHON" ]]], axis = 1)

color_map = {
    "T_interface": "darkblue", 
    "T_interface_PYTHON": "blue", 
    "T_ins": "darkgreen", 
    "T_insulation_PYTHON": "green" 
}

import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(rows=1, cols=2, subplot_titles=("Interface layer", "Insulation layer"))

color_map = {
    "T_interface": "darkblue", 
    "T_interface_PYTHON": "blue", 
    "T_ins": "darkgreen", 
    "T_insulation_PYTHON": "green" 
}
for col in ["T_interface", "T_interface_PYTHON"]:
    fig.add_trace(
        go.Scatter(x=measure_comp.index, y=measure_comp[col], mode="lines",
                   name=col, line=dict(color=color_map[col])),
        row=1, col=1
    )
for col in ["T_ins", "T_insulation_PYTHON"]:
    fig.add_trace(
        go.Scatter(x=measure_comp.index, y=measure_comp[col], mode="lines",
                   name=col, line=dict(color=color_map[col])),
        row=1, col=2
    )
fig.update_layout(
    title="Comparaison Python vs Measurement",
    width=1000,
    height=500,
    legend=dict(orientation="h", yanchor="bottom", y=-0.2, xanchor="center", x=0.5)
)

fig.show()

Not  good. A sensitivity analysis should be performed to rank the parameters by order of influence on the error between measured temperature and model prediction.


# 3. Sensitivity analysis
It is very important to know how our defined parameters have an influence on the model prediction. Therefore, we use a sensitivity analysis to "rank" the parameter by order of influence
on the model error. 

## Error function
The chosen error function is the CV_RMSE. The formula for CV_RMSE is given by:

$$
CV\_RMSE = \frac{RMSE}{\bar{y}}
$$

Where:
- *RMSE* is the root mean squared error,
- *bar{y}* is the mean of the observed values.

The RMSE is calculated as:

$$
RMSE = \sqrt{\frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2}
$$

Where:
- *n* is the number of observations,
- *y_i* is the observed value for the \( i \)-th observation,
- *hat{y}_i* is the predicted value for the \( i \)-th observation.

The CV_RMSE measures the variation of the RMSE relative to the mean of the observed values. It provides a standardized measure of the error, which can be useful for comparing the performance of different models across different datasets.
Here, we can chose the error function as the CV_RMSE between measured temperature(s) and model prediction.

## Tested parameters

The chosen parameters are the following model parameters : R_concrete, R_ins, C_ins, C_concrete, alpha, epsilon, R_ext, R_int.


Each decision variable of the optimization problem must be described using a `Parameter` object.
A parameter specifies:
* `name` — The variable name (string, must be unique within the problem).
* `ptype` — Variable type, one of:
    * `"Real" `— Continuous real number
    * `"Integer"` — Discrete integer
    * `"Binary"` — Boolean, domain {False, True} (set automatically if no domain is given)
    *` "Choice"` — Categorical variable with a fixed set of discrete options
* `Domain definition` — Choose exactly one of:
    * ` interval=(lo, hi) `— Lower and upper bounds (for "Real" and "Integer", optional for "Binary" if you want (0,1))
    * ` values=(v1, v2, …)` — Explicit list/tuple of allowed values (for "Choice", and optionally for "Integer", "Real", or "Binary")
*` Optional fields`:
    * `init_value` — Initial value (or tuple/list of initial values for batch runs); must be within the defined domain
    * `relabs` — `"Absolute"` or `"Relative"` (or a boolean flag, depending on usage in your model)
    * `model_property` — String or tuple specifying the corresponding property in the simulation/model
    * `min_max_interval` — Optional extra bounds for use in analysis/validation

In [ ]:
from corrai.base.parameter import Parameter

In [ ]:
frac_p = 0.40 # 20%
frac_conv = 0.60 # 40%

params = [
    Parameter(
        name='R_concrete',
        interval=(1-frac_p, 1+frac_p),
        ptype="Real",
        init_value=1 / (lambda_concrete / ep_concrete) / 2 / S_wall,
        relabs="Relative",
        model_property='R_concrete',
    ),
    Parameter(
        name='R_ins',
        interval=(1-frac_p, 1+frac_p),
        init_value=1 / (lambda_ins / ep_ins) / 2 / S_wall,
        relabs="Relative",
        ptype="Real",
        model_property='R_ins',
    ),
    Parameter(
        name='C_ins',
        interval=(1-frac_p, 1+frac_p),
        init_value=rho_ins*ep_ins*S_wall*sc_ins,
        relabs="Relative",
        ptype="Real",
        model_property='C_ins',
    ),
    Parameter(
        name='C_concrete',
        interval=(1-frac_p, 1+frac_p),
        init_value=rho_concrete*ep_concrete*S_wall*sc_concrete,
        relabs="Relative",
        ptype="Real",
        model_property='C_concrete',
    ),
    Parameter(
        name='alpha',
        interval=(0.1, 0.6),
        ptype="Real",
        model_property='alpha',
    ),
    Parameter(
        name='epsilon',
        interval=(0.2, 0.9),
        ptype="Real",
        model_property='epsilon',
    ),
    Parameter(
        name='R_ext',
        init_value= 0.04/S_wall,
        interval=(1-frac_conv, 1+frac_conv),
        ptype="Real",
        relabs="Relative",
        model_property='R_ext',
    ),
    Parameter(
        name='R_int',
        init_value= 0.13/S_wall,
        interval=(1-frac_conv, 1+frac_conv),
        ptype="Real",
        relabs="Relative",
        model_property='R_int',
    ),
]

## Sensitivity analysis methods
We can now load from `corrai.sensitivity` module a sensitivity method.

*Note: for now, only <code>SOBOL</code>, <code>FAST</code>, <code>RBD_FAST</code>, 
and <code>MORRIS</code> methods are implemented.*

### MORRIS method

We should start off with a screening using <code>MORRIS</code> method.

For <code>MORRIS</code> method, two indices, µj* for the mean of the absolute values of these effects and σj for the standard deviation of these effects, are calculated as follows:

$$
mu_{j}^{*} = \frac{1}{r} \sum_{i=1}^{r} E_{ij}
$$

$$
sigma_{j} = \sqrt{\frac{1}{r-1} \sum_{i=1}^{r} (E_{ij} - \mu_{j}^{*})^2}
$$

Let's resample the data to 5 minute-steps. 

In [ ]:
simulation_df_resample = simulation_df.resample("5min").mean()
simulation_df_resample["time_sec"] = datetime_to_seconds(simulation_df_resample.index)

sim_opt ={
    "dataframe":simulation_df_resample,
    "startTime":simulation_df_resample["time_sec"].iloc[0],
    "endTime": simulation_df_resample["time_sec"].iloc[-1],  
}

In [ ]:
from corrai.sensitivity import MorrisSanalysis

sa_study = MorrisSanalysis(
    parameters=params,
    model=model,
    simulation_options=sim_opt,
)

In [ ]:
sa_study.add_sample(N=2**6, n_cpu=-1)

First, we can plot all simulations in one graph and compare the simulated internal temperature to measured T_int. Argument <code>show_legends</code> can be set to True if you want see associated parameters values.

In [ ]:
sa_study.sampler.plot_sample(
    indicator="T_ins",
    reference_timeseries = simulation_df_resample["T_ins"],
    x_label="time",
    y_label="simulated temperature of T_ins [°C]",
    show_legends=False
)

First, we can first take a look on the parallel coordinate plot of all parameter values and one of the simulation outputs aggregated according to a chosen aggregation method:This graph can be very instructive as at some moments, simulations are far from measurements. It show that whatever the values of our parameters, it still does not fit reality: this is either due to a problem of measurement, or to our modeling approach (physical inconsistency, physical phenomenon not properly taken into account, etc.).


Let's perform a **Morris sensitivity analysis** on the following indicator: cv_rmse on `T_ins`. 
The `analyze()` method computes Sobol indices based on a chosen performance metric—in this case, the **cv_rmse**—by comparing model predictions of `T_ins` against the provided reference time series (measured or baseline data).

In [ ]:
res_analysis = sa_study.analyze(
    indicator="T_ins",
    method="cv_rmse",
    reference_time_series=simulation_df_resample["T_ins"],
)
res_analysis["cv_rmse_T_ins"]

The highter $mu_{j}$, the more the parameter $j$ contributes to an uncertain output, and the higher $sigma_{j}$, the more pronounced the interaction effects between the model parameters are. Plotting $sigma_{j}$ against $mu_{j}^{}$ is often used to distinguish factors with negligible, linear, and/or interaction effects.

In [ ]:
sa_study.plot_scatter(
    indicator="T_ins",
    method="cv_rmse",
    reference_time_series=simulation_df_resample["T_ins"],
)

Besides this visual interpretation, it is also possible to calculate the Euclidean distance $d$ to the origin to obtain the total effect of the uncertain parameters:

In [ ]:
sa_study.plot_bar(
    indicator="T_ins",
    method="cv_rmse",
    reference_time_series=simulation_df_resample["T_ins"],
)

Five parameters seem to have more impact on the error than other: $alpha$, $R_{ext}$, $R_{ins}$,  and $R_{concrete}$. 
Let's check if these results are consistant with <code>SOBOL</code> method.

### SOBOL method

In [ ]:
from corrai.sensitivity import SobolSanalysis

In [ ]:
sa_study = SobolSanalysis(
    parameters=params,
    model=model,
    simulation_options=sim_opt,
)

The method add sample draw a sample of parameters to be simulated. Each method has its sampling method. Please see SALib documentation for further explanation (https://salib.readthedocs.io/en/latest/index.html)

Note that:
- Convergence properties of the Sobol' sequence is only valid if
        `N` (100) is equal to `2^n`.
        N (int) – The number of samples to generate. Ideally a power of 2 and <= skip_values.
- Convergence properties of the Fast' method is only valid if sample size N > 4M^2 (M=4 by default)

In [ ]:
sa_study.add_sample(N=2**7,  n_cpu=-1)

As for MORRIS, we can perform a **Sobol sensitivity analysis** on the following indicator: cv_rmse on `T_ins` .

In [ ]:
res_analysis =sa_study.analyze(
    indicator="T_ins",
    method="cv_rmse",
    reference_time_series=simulation_df_resample["T_ins"],
)
print("Sum of S1 indices is", res_analysis["cv_rmse_T_ins"]["S1"].sum())

The `plot_bar()` method displays the Sobol sensitivity indices as a bar chart, allowing a quick comparison of parameter importance.
It uses the aggregated results from the `analyze()` step (e.g., based on `cv_rmse`) and shows  first-order indices for each parameter, making it easy to identify which parameters most influence the chosen performance metric.

The sum of all the indices should be close to 1. Also, the mean confidence interval should be very low. In that case, results of the sensitivity analysis can be considered as robust.

In [ ]:
sa_study.plot_bar(
    indicator="T_ins",
    method="cv_rmse",
    reference_time_series=simulation_df_resample["T_ins"],
)

The parameter alpha appears to have the most influence on the model errors. This impact is calculated on the overall error, but will depend on the time of the day. Let's observe the parameters' impact dynamically with a 15minutes frequency.

The `plot_dynamic_metric()` method extends the analysis to a frequency view of the selected performance metric.
Here, we apply it to `T_ins` using the `cv_rmse` metric, comparing model output against the `reference_time_series` at regular intervals.

Key arguments:
- **`freq`**: Controls the temporal resolution for aggregation (e.g., `freq="2h"` computes sensitivity indices every two hours). This enables tracking how parameter influence changes over time.
- **`method`**: The metric used for comparison (here, `cv_rmse` between model predictions and the reference).
- **`reference_time_series`**: The baseline data against which each simulation is evaluated.

This approach is particularly useful in dynamic systems, where parameter importance may vary significantly across different time periods.

*Calculation can take time according to the number of simulations and frequency of data.*


In [ ]:
sa_study.plot_dynamic_metric("T_ins", freq="2h", method="cv_rmse", reference_time_series=simulation_df_resample["T_ins"] )

Finally, we can also take a look at the parallel coordinate plot of all parameter values and one of the simulation outputs aggregated according to a chosen aggregation method. This is a first step to foresee correct values of parameters to fit the measurement.

In [ ]:
sa_study.plot_pcp(
    indicator="T_ins" ,
    method="cv_rmse",
    reference_time_series=simulation_df_resample["T_ins"],
)

# Conclusion on sensitivity analysis

The sensitivity analysis allowed us to rank the influence of uncertain parameters
on an indicator. 

Results with Sobol are consistant with Morris. $alpha$ is the most influencial parameters, followed by $R_{ext}$.

In the Identification tutorial, modules of corrai are used to identify the
optimal values for these parameters in order to fit the measurement. See : https://github.com/BuildingEnergySimulationTools/corrai/tutorials/Sensitivityanalysis for more details.